# Generic AST Visualization

While reading nested tuples is possible, it is cognitively demanding. To inspect our parser's output efficiently, we implement a **generic visualizer** that transforms our AST into a Directed Acyclic Graph (DAG) using the DOT language.

## Mathematical Transformation

We define a transformation function $V$ that maps an AST node $n$ and a parent context $p$ to a set of graph definitions (nodes and edges).

Let $\mathcal{A}$ be the set of all AST nodes. The transformation $V: \mathcal{A} \times ID \rightarrow DOT$ follows three rules:

1.  **Terminals (Leaves):**
    If $n$ is a number or string, it is a leaf. We draw a node and an edge from the parent $p$.
    $$
    V(l, p) = \{ \text{Node}(id_l, \text{label}=l), \text{Edge}(p \to id_l) \}
    $$

2.  **Cons-List Flattening (The "Invisible" Node):**
    Our AST uses Lisp-style cons-cells `['.', head, tail]` for lists. Visually, drawing every dot would create deep, right-leaning ladders. Instead, we apply a **transparency optimization**: we treat the `.` node as invisible and attach its children directly to the grandparent $p$.
    $$
    V([\text{'.'}, h, t], p) = V(h, p) \cup V(t, p)
    $$
    This "flattens" the visual representation of sequences.

3.  **Standard Nodes:**
    For any other operation $[Tag, c_1, \dots, c_k]$, we create a distinct node and recursively visit children with the new node as their parent.
    $$
    V([Tag, \dots], p) = \{ \text{Node}(id_{new}, Tag), \text{Edge}(p \to id_{new}) \} \cup \bigcup_{i} V(c_i, id_{new})
    $$

## Visual Semantics

To make the graph readable at a glance, we assign semantic shapes and colors:
*   **Green Boxes:** Blocks `{ ... }` (Scopes)
*   **Red Octagons:** Function Calls `Call(...)`
*   **Blue Diamonds:** Control Flow (`IF`, `WHILE`)
*   **Circles:** Operators

In [ ]:
function ast2dot(input: unknown): string {
    let nodeCount = 0;
    const lines: string[] = [
        "digraph AST {",
        '  node [shape=box, style=filled, fontname="Courier", fontsize=10];',
        '  edge [arrowsize=0.7];'
    ];

    function visit(thing: unknown, parentId: string | null = null): void {
        if (thing === null || thing === "") return;

        // 1. Terminals
        if (typeof thing === "string" || typeof thing === "number") {
            const id = `n${nodeCount++}`;
            const label = thing.toString().replace(/"/g, '\\\\"');
            lines.push(`  ${id} [label="${label}", shape=ellipse, fillcolor="#fffbe6", margin=0.05];`);
            
            if (parentId) {
                lines.push(`  ${parentId} -> ${id};`);
            }
            return;
        }

        // 2. Composite Nodes
        if (Array.isArray(thing)) {
            const [tag, ...children] = thing;

            // --- Invisible List Node ---
            if (tag === '.') {
                const [head, tail] = children;
                visit(head, parentId);
                visit(tail, parentId);
                return;
            }

            // --- Standard Nodes ---
            const id = `n${nodeCount++}`;
            
            let label = tag;
            let shape = "box";
            let color = "#eeeeee";

            if (tag === "block") {
                label = "BLOCK"; 
                color = "#e6ffe6"; 
            } else if (tag === "Call") {
                shape = "octagon";
                color = "#ffe6e6"; 
            } else if (["IF", "WHILE", "FOR"].includes(tag)) {
                shape = "diamond";
                color = "#e6f2ff"; 
            } 
            // Hier habe ich ^, % und die Vergleiche ergänzt:
            else if ([":=", "+", "-", "*", "/", "%", "^", "<", ">", "<=", ">=", "==", "!="].includes(tag)) {
                shape = "circle";
                label = tag; 
            }

            lines.push(`  ${id} [label="${label}", shape="${shape}", fillcolor="${color}"];`);

            if (parentId) {
                lines.push(`  ${parentId} -> ${id};`);
            }

            for (const child of children) {
                visit(child, id);
            }
        }
    }

    visit(input);
    lines.push("}");
    return lines.join("\n");
}